# Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
import timm
from tqdm import tqdm
import pandas as pd

# Device and Constants

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = 224
mean = (0.2860, 0.2860, 0.2860)
std = (0.3530, 0.3530, 0.3530)
batch_size = 16

# Transforms

In [ ]:
def make_train_transform(img_size):
    return T.Compose([
        T.Resize(img_size),
        T.Grayscale(num_output_channels=3),
        T.RandomHorizontalFlip(),
        T.RandomCrop(img_size, padding=4),
        T.ToTensor(),
        T.Normalize(mean, std)
    ])

def make_test_transform(img_size):
    return T.Compose([
        T.Resize(img_size),
        T.Grayscale(num_output_channels=3),
        T.ToTensor(),
        T.Normalize(mean, std)
    ])

# Data

In [ ]:
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=make_train_transform(img_size))
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=make_test_transform(img_size))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

#  Model (Swin Tiny)

In [ ]:
model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 10)
model = model.to(device)

# Optimizer, Scheduler, Loss

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.3)
criterion = nn.CrossEntropyLoss()
epochs = 10

#  Training/Evaluation Functions

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        if outputs.dim() == 4:
            outputs = outputs.mean(dim=(1, 2))
        elif outputs.dim() == 3:
            outputs = outputs.mean(1)       
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        running_loss += loss.item() * images.size(0)
    avg_loss = running_loss / total
    avg_acc = 100.0 * correct / total
    return avg_loss, avg_acc

def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if outputs.dim() == 4:
                outputs = outputs.mean(dim=(1, 2))
            elif outputs.dim() == 3:
                outputs = outputs.mean(1)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100.0 * correct / total

# Training Loop

In [ ]:
for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(model, trainloader, criterion, optimizer, device)
    test_acc = evaluate(model, testloader)
    print(f"Epoch {epoch+1}/{epochs}: "
          f"Train Loss: {train_loss:.4f}, "
          f"Train Acc: {train_acc:.2f}%, "
          f"Test Acc: {test_acc:.2f}%")
    scheduler.step()
    torch.cuda.empty_cache()


#  Save Model

In [ ]:
torch.save(model.state_dict(), "Swin-Tiny_FashionMNIST.pth")
print("Saved Swin-Tiny trained weights.")

In [ ]:
from torchvision import transforms as T

def make_transforms(img_size):
    mean = (0.2860, 0.2860, 0.2860)
    std = (0.3530, 0.3530, 0.3530)
    return {
        "Clean": T.Compose([
            T.Resize(img_size),
            T.Grayscale(num_output_channels=3),
            T.ToTensor(),
            T.Normalize(mean, std),
        ]),
        "Horizontal Flip": T.Compose([
            T.Resize(img_size),
            T.Grayscale(num_output_channels=3),
            T.RandomHorizontalFlip(p=1.0),
            T.ToTensor(),
            T.Normalize(mean, std),
        ]),
        "Rotation": T.Compose([
            T.Resize(img_size),
            T.Grayscale(num_output_channels=3),
            T.RandomRotation(30),
            T.ToTensor(),
            T.Normalize(mean, std),
        ]),
        "Blur": T.Compose([
            T.Resize(img_size),
            T.Grayscale(num_output_channels=3),
            T.GaussianBlur(3),
            T.ToTensor(),
            T.Normalize(mean, std),
        ]),
        "Brightness": T.Compose([
            T.Resize(img_size),
            T.Grayscale(num_output_channels=3),
            T.ColorJitter(brightness=0.5),
            T.ToTensor(),
            T.Normalize(mean, std),
        ]),
        "Gaussian Noise": T.Compose([
            T.Resize(img_size),
            T.Grayscale(num_output_channels=3),
            T.ToTensor(),
            T.Lambda(lambda x: x + 0.15 * torch.randn_like(x)),
            T.Normalize(mean, std),
        ]),
    }

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = 224


model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=False)
model.head = torch.nn.Linear(model.head.in_features, 10)
model.load_state_dict(torch.load('/kaggle/working/Swin-Tiny_FashionMNIST.pth', map_location=device))
model = model.to(device)
model.eval()


curr_transforms = make_transforms(img_size)
results = {}

for name, transform in curr_transforms.items():
    dataset = torchvision.datasets.FashionMNIST(
        root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=False, num_workers=2)

    def evaluate(model, dataloader):
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in dataloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                if outputs.dim() == 4:
                    outputs = outputs.mean(dim=(1,2))
                elif outputs.dim() == 3:
                    outputs = outputs.mean(1)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        return 100.0 * correct / total

    acc = evaluate(model, testloader)
    results[name] = acc
    print(f"{name}: {acc:.2f}%")

df = pd.DataFrame([results], index=['Swin-Tiny'])
display(df.T.style.background_gradient(cmap='Blues'))